#### Notebook 2
This notebook extracts the features from the test images using an inceptionv3 model with the imagenet weights.

In [1]:
import keras


Using TensorFlow backend.


In [2]:
# keras imports
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from PIL import Image

In [3]:
# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import PIL


In [4]:
# load the user configs
with open('room_recognition/conf/conf_inceptionv3_getfeatures.json') as f:    
  config = json.load(f)

In [5]:
# config variables
model_name  = config["model"]
weights     = config["weights"]

test_path  =config["test_path"]
test_features_path   = config["test_features_path"]
test_labels_path   = config["test_labels_path"]
seed =config["seed"]





In [6]:
# start time
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()

[STATUS] start time - 2018-09-20 23:02


In [7]:
# create the pretrained models
base_model = InceptionV3(include_top=True, weights=weights, input_tensor=Input(shape=(299,299,3)))
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
image_size = (299, 299)

print ("[INFO] successfully loaded base model and model...")

[INFO] successfully loaded base model and model...


In [8]:
# path to training dataset
test_labels = os.listdir(test_path)

In [9]:
# encode the labels
print ("[INFO] encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in test_labels])


[INFO] encoding labels...


LabelEncoder()

In [10]:
# variables to hold features and labels
features = []
labels   = []

In [11]:
# loop over all the labels in the folder
count = 1
for i, label in enumerate(test_labels):
  cur_path = test_path + "/" + label
  count = 1
  for image_path in glob.glob(cur_path + "/*.jpg"):
    img = image.load_img(image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    features.append(flat)
    labels.append(label)
    print ("[INFO] processed - " + str(count))
    count += 1
  print ("[INFO] completed label - " + label)


[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[INFO] processed - 15
[INFO] processed - 16
[INFO] processed - 17
[INFO] processed - 18
[INFO] processed - 19
[INFO] processed - 20
[INFO] completed label - bathroom
[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] processed - 4
[INFO] processed - 5
[INFO] processed - 6
[INFO] processed - 7
[INFO] processed - 8
[INFO] processed - 9
[INFO] processed - 10
[INFO] processed - 11
[INFO] processed - 12
[INFO] processed - 13
[INFO] processed - 14
[INFO] processed - 15
[INFO] processed - 16
[INFO] processed - 17
[INFO] processed - 18
[INFO] processed - 19
[INFO] processed - 20
[INFO] completed label - pantry
[INFO] processed - 1
[INFO] processed - 2
[INFO] processed - 3
[INFO] pr

In [12]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

In [13]:
# get the shape of training labels
print ("[STATUS] test labels: {}".format(le_labels))
print ("[STATUS] test labels shape: {}".format(le_labels.shape))

[STATUS] test labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7]
[STATUS] test labels shape: (160,)


In [14]:
# save features and labels
h5f_data = h5py.File(test_features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(test_labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

In [15]:
# end time
end = time.time()
print ("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] end time - 2018-09-20 23:03
